<a href="https://colab.research.google.com/github/faziletgokbudak/ESPCN-with-perceptual-loss/blob/master/DetailTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps

In [ ]:
def compute_laplacian_pyramid(img_path, num_levels):
  '''
  Computes the Laplacian pyramid of an image in img_path with the number of 
  levels defined by num_levels.
  '''
  # Load the image
  img = cv2.imread(img_path)
  lower = img.copy()
  # Create a Gaussian Pyramid
  gaussian_pyr = [lower]

  for i in range(num_levels):
      lower = cv2.pyrDown(lower)
      gaussian_pyr.append(lower)
  
  # Last level of Gaussian remains same in Laplacian
  laplacian_top = gaussian_pyr[-1]
  
  # Compute a Laplacian Pyramid
  laplacian_pyr = [laplacian_top]
  for i in range(num_levels, 0, -1):
      size = (gaussian_pyr[i-1].shape[1], gaussian_pyr[i-1].shape[0])
      gaussian_expanded = cv2.pyrUp(gaussian_pyr[i], dstsize=size)
      laplacian = cv2.subtract(gaussian_pyr[i-1], gaussian_expanded)
      laplacian_pyr.append(laplacian)
      cv2_imshow(laplacian)
  return laplacian_pyr

laplacian_pyr = compute_laplacian_pyramid('apple.jpg', 5)
print(len(laplacian_pyr))

In [ ]:
#image instead of path
#input output all tensors
def extract_image_patches(image_path, patch_dims, stride):
  '''
  Extracts the image patches of dimension, patch_dims, with a stride of integer 
  number.
  patch_dims: 2D list
  '''
  gray_image = tf.dtypes.cast(tf.convert_to_tensor(np.asarray(
      ImageOps.grayscale(image))), dtype = tf.float32)

  tensor_img = tf.expand_dims(gray_image, axis=0) #extra dim for batches
  tensor_img2 = tf.expand_dims(tensor_img, axis=-1) #extra dim for color channels

  patches = tf.image.extract_patches(images=tensor_img2,
                          sizes=[1, patch_dims[0], patch_dims[1], 1],
                          strides=[1, stride, stride, 1],
                          rates=[1, 1, 1, 1],
                          padding='VALID')
  
  squeezed_patches = tf.squeeze(patches, axis=0) 

  reshaped_squeezed_patches = tf.reshape(squeezed_patches, 
                                         [squeezed_patches.shape[0]*
                                          squeezed_patches.shape[1],
                                          patch_dims[0], patch_dims[1]])
  return reshaped_squeezed_patches

In [ ]:
def reconstruct_image_from_patches(patches, img_dims, stride):
  '''
  Combines the patches and saves the reconstructed image
  '''
  new_image = np.array(tf.zeros([img_dims[0], img_dims[1]], dtype=tf.dtypes.float32, name=None))
  merging_map = np.array(tf.zeros([img_dims[0], img_dims[1]], dtype=tf.dtypes.float32, name=None))

  count = 0
  for i in range(0, img_dims[0]-patch_dims[0]+1, stride):
    for j in range(0, img_dims[1]-patch_dims[1]+1, stride):
      # check the guideline!!!!
      new_image[i:i+patch_dims[0], j:j+patch_dims[1]] = new_image[i:i+patch_dims[0], j:j+patch_dims[1]] + reshaped_squeezed_patches[count][:][:]
      merging_map[i:i+patch_dims[0], j:j+patch_dims[1]] += 1
      count += 1
      #print(count)

  image_new = np.rint(np.divide(new_image, merging_map)) 
  im = Image.fromarray(((image_new)))
  new_p = im.convert("L").save('reconstructed.jpg')
  new_image_tensor = tf.convert_to_tensor(new_image)
  return new_image_tensor